In [ ]:
from canns.task.tracking import PopulationCoding1D
from canns.models.basic import CANN1D
import brainpy.math as bm  # :cite:p:`wang2023brainpy`

# First create model instance
bm.set_dt(0.1)
model = CANN1D(num=256, tau=1.0, k=8.1, a=0.5, A=10, J0=4.0)

# Create task
task = PopulationCoding1D(
    cann_instance=model,      # CANN model instance
    before_duration=10.0,     # Duration before stimulus
    after_duration=50.0,      # Duration after stimulus
    Iext=1.0,                 # Stimulus position in feature space
    duration=10.0,            # Stimulus duration
    time_step=0.1,            # Time step
)

In [2]:
# Generate task data
task.get_data()

# Access task properties
print(f"Total time steps: {task.total_steps}")
print(f"Total duration: {task.total_duration}")
print(f"Data shape: {task.data.shape}")

<PopulationCoding1D>Generating Task data(No For Loop)
Total time steps: 700
Total duration: 70.0
Data shape: (700, 256)


In [3]:
import brainpy.math as bm  # :cite:p:`wang2023brainpy`

# Define step function
def run_step(t, inp):
    """
    Single simulation step.

    Args:
        t: Current time step index
        inp: Input data at current time step

    Returns:
        State variables to record
    """
    model(inp)  # Or model.update(inp)
    return model.u.value, model.r.value

# Run simulation using task.data
results = bm.for_loop(
    run_step,           # Step function
    operands=(task.run_steps, task.data),  # Number of time steps and input data
    progress_bar=10  # Optional progress bar (updates every 10 steps)
)

  0%|          | 0/700 [00:00<?, ?it/s]

In [4]:
# results is a tuple of return values across all time steps
u_history, r_history = results

print(f"Membrane potential history shape: {u_history.shape}")  # (run_steps, num)
print(f"Firing rate history shape: {r_history.shape}")  # (run_steps, num)

Membrane potential history shape: (700, 256)
Firing rate history shape: (700, 256)


In [9]:
import time

model = CANN1D(num=256, tau=1.0, k=8.1, a=0.5, A=10, J0=4.0)

# Create a new longer task
task = PopulationCoding1D(
    cann_instance=model,      # CANN model instance
    before_duration=10000.0,     # Duration before stimulus
    after_duration=10000.0,      # Duration after stimulus
    Iext=1.0,                 # Stimulus position in feature space
    duration=10000.0,            # Stimulus duration
    time_step=0.1,            # Time step
)
task.get_data()

# First run (includes compilation)
start = time.time()
results = bm.for_loop(run_step, operands=(task.run_steps, task.data))
print(f"First run: {time.time() - start:.2f}s")

# Second run (already compiled)
start = time.time()
results = bm.for_loop(run_step, operands=(task.run_steps, task.data))
print(f"Second run: {time.time() - start:.2f}s")

<PopulationCoding1D>Generating Task data(No For Loop)
First run: 0.75s
Second run: 0.24s


In [10]:
import brainpy.math as bm  # :cite:p:`wang2023brainpy`
from canns.models.basic import CANN1D
from canns.task.tracking import PopulationCoding1D

# ============================================================
# Step 1: Setup environment and create model
# ============================================================
bm.set_dt(0.1)

model = CANN1D(num=256, tau=1.0, k=8.1, a=0.5, A=10, J0=4.0)

# ============================================================
# Step 2: Create task
# ============================================================
task = PopulationCoding1D(
    cann_instance=model,
    before_duration=10.0,
    after_duration=50.0,
    Iext=0.0,
    duration=10.0,
    time_step=0.1,
)

# Get task data
task.get_data()

print("Task Information:")
print(f"  Total time steps: {task.total_steps}")
print(f"  Total duration: {task.total_duration}")
print(f"  Data shape: {task.data.shape}")

# ============================================================
# Step 3: Define simulation step function
# ============================================================
def run_step(t, inp):
    model.update(inp)
    return model.u.value, model.r.value

# ============================================================
# Step 4: Run simulation
# ============================================================
u_history, r_history = bm.for_loop(
    run_step,
    operands=(task.run_steps, task.data),
)

# ============================================================
# Step 5: Inspect results
# ============================================================
print("\nSimulation Results:")
print(f"  Membrane potential history shape: {u_history.shape}")
print(f"  Firing rate history shape: {r_history.shape}")

# Check states at different phases
before_steps = int(10.0 / 0.1)  # Before stimulus
stim_end = int(20.0 / 0.1)      # End of stimulus
after_steps = int(70.0 / 0.1)   # End of simulation

print(f"\nBefore stimulus (t={before_steps-1}) max firing rate: {bm.max(r_history[before_steps-1]):.6f}")
print(f"During stimulus (t={stim_end-1}) max firing rate: {bm.max(r_history[stim_end-1]):.6f}")
print(f"After stimulus (t={after_steps-1}) max firing rate: {bm.max(r_history[after_steps-1]):.6f}")

<PopulationCoding1D>Generating Task data(No For Loop)
Task Information:
  Total time steps: 700
  Total duration: 70.0
  Data shape: (700, 256)

Simulation Results:
  Membrane potential history shape: (700, 256)
  Firing rate history shape: (700, 256)

Before stimulus (t=99) max firing rate: 0.000000
During stimulus (t=199) max firing rate: 0.002426
After stimulus (t=699) max firing rate: 0.002348
